# Applying PCA, then regressing

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
ms = pd.read_feather("./0407_pca_2/measures.feather")
ms["model.pca_dims"] = ms["model.pca_dims"].fillna(0)

smooth.analysis.remove_constant_columns(ms, verbose=True)

ms.head()

In [ ]:
def plot_measure(measure_name):
    counted_col = "model.pca_dims"
    dimss = sorted(ms[counted_col].unique())
    data = ms[[measure_name, counted_col]].copy()
    
    if "loss" in measure_name:
        log_measure_name = "log10_{}".format(measure_name)
        data[log_measure_name] = np.log10(ms[measure_name])
        measure_name = log_measure_name
    
    for dims in dimss:
        grid = sns.distplot(
            data.loc[data[counted_col] == dims, measure_name],
            label=dims,
            hist_kws={
                "range": (data[measure_name].min(), data[measure_name].max()),
            },
            bins=20,
        )

    plt.legend()
    plt.show()

for measure in [
            "loss_train", "loss_test",
            "gradient_norm_test",
            "weights_product",
            "path_length_f_test",
            "path_length_d_test",
        ]:
    plot_measure(measure)

In [ ]:
import sklearn.decomposition

dataset = smooth.datasets.from_params("mnist01")
pca = sklearn.decomposition.PCA(n_components=28*28)
x_train = dataset.x_train.reshape(len(dataset.x_train), -1)
pca.fit(x_train)

In [ ]:
pca.transform(x_train[:1])

In [ ]:
np.linalg.norm(x_train[:1] - pca.mean_)

In [ ]:
np.linalg.norm(pca.transform(x_train[:1]))

In [ ]:
np.dot(pca.components_, (x_train[0] - pca.mean_))

In [ ]:
layer = tf.keras.layers.Dense(pca.n_components, weights=[pca.components_.T, -np.dot(pca.components_, pca.mean_)])

In [ ]:
layer(x_train[:1])

In [ ]:
pca.transform(x_train[:1])

In [ ]:
model = smooth.model.get_shallow(dataset, init_scale=1., hidden_size=20, activation="relu", pca_dims=10)